### M.A.C Overview

__Moving Average Crossover Strategy: Overview and Market Conditions__

The moving average crossover strategy is a widely used technical analysis tool for identifying potential buy and sell signals in financial markets. This strategy involves plotting two moving averages of different time periods—a shorter-term and a longer-term—and analyzing their interactions. When the shorter moving average crosses above the longer one, it generates a bullish signal ("golden cross"), while the opposite generates a bearish signal ("death cross").

This document summarizes the optimal market conditions for using the moving average crossover strategy, its limitations, and specific Forex pairs that align with these scenarios.

---

__Optimal Market Conditions__

1. __Trending Markets__
   - **Bullish Trends**: The strategy works well in upward-trending markets. A "golden cross" signals potential entry points for long positions.
     - **Examples**: 
       - **USD/JPY**: Often trends upward during U.S. economic strength or rising interest rates.
       - **AUD/USD**: Reflects strong bullish trends during favorable commodity market conditions.
   - **Bearish Trends**: Downward-trending markets provide opportunities for short positions when a "death cross" occurs.
     - **Examples**: 
       - **EUR/USD**: Clear downward trends appear during eurozone economic challenges.
       - **GBP/USD**: Exhibits bearish trends during political instability or negative Brexit developments.

2. __Low Volatility Markets__
   - The strategy performs better in low to moderate volatility conditions, avoiding frequent false signals.
     - **Examples**: 
       - **EUR/CHF**: Relatively stable due to eurozone-Switzerland economic correlation.
       - **USD/SGD**: Features consistent, low-volatility price movements absent major news events.

3. __Markets with Consistent Trends and Cycles__
   - Pairs that exhibit cyclical price behavior are well-suited for this strategy.
     - **Examples**: 
       - **AUD/JPY**: Cycles tied to global risk sentiment.
       - **NZD/USD**: Moves in patterns driven by agricultural exports and commodity markets.

---

__Limitations of Moving Average Crossovers__

1. __Sideways or Range-Bound Markets__
   - In choppy or range-bound conditions, the strategy often generates false signals, leading to losses.
     - **Examples**: 
       - **EUR/GBP**: Commonly trades within a range due to economic interdependence.
       - **USD/CAD**: Becomes range-bound during stable oil prices and economic parity.

2. __High-Volatility Markets__
   - Excessive price swings in volatile markets can produce unreliable signals, causing whipsaws.
     - **Examples**: 
       - **GBP/JPY**: Known for erratic and high-volatility price movements.
       - **XAU/USD (Gold)**: Frequently experiences sharp price fluctuations, making crossovers less reliable.

---

__Best Practices for Backtesting__

- **Trending Pairs**: Use pairs like **EUR/USD** or **USD/JPY** to test the effectiveness of the strategy in trending conditions.
- **Volatility Management**: Avoid highly volatile pairs like **GBP/JPY** or commodities like **XAU/USD** when testing pure crossover strategies.
- **Combine Indicators**: Pair moving average crossovers with complementary technical tools (e.g., RSI, MACD) to filter false signals and enhance decision-making.

---

This summary provides a foundational understanding of the moving average crossover strategy, enabling informed parameter selection and context-specific application during backtesting.



### Initialize and Login

In [36]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from datetime import datetime
import calendar
from tqdm import tqdm

# For Charts
import plotly.graph_objects as go

# To access .env file
from dotenv import load_dotenv

# For random selection when TP and SL hit simultaneously
import random  


import os
import time
import talib
from talib import abstract

In [37]:
# Access .env file
load_dotenv(".env")

# Access variables directly
username = int(os.getenv("USER_NAME"))
password = os.getenv("PASSWORD")
server = os.getenv("SERVER")

symbols = os.getenv("SYMBOLS")

In [38]:
# Initialize and log in to MetaTrader 5
# connect to MetaTrader 5
try:
    # Initialize MetaTrader 5
    if mt5.initialize():
        print("MetaTrader successfully initialized")
        
        # Log in to MetaTrader 5
        if mt5.login(login=username, password=password, server=server):
            print("MetaTrader login successful")
                
        else:
            print("MetaTrader login failed")
            mt5.shutdown()  # Ensure shutdown if login fails
    else:
        print("Failed to initialize MetaTrader 5")
except Exception as e:
    print(f"An error occurred: {e}")

MetaTrader successfully initialized
MetaTrader login successful


### TA-Library

In [31]:
list(talib.get_function_groups().keys())

['Cycle Indicators',
 'Math Operators',
 'Math Transform',
 'Momentum Indicators',
 'Overlap Studies',
 'Pattern Recognition',
 'Price Transform',
 'Statistic Functions',
 'Volatility Indicators',
 'Volume Indicators']

In [32]:
talib.get_function_groups()['Momentum Indicators']

['ADX',
 'ADXR',
 'APO',
 'AROON',
 'AROONOSC',
 'BOP',
 'CCI',
 'CMO',
 'DX',
 'MACD',
 'MACDEXT',
 'MACDFIX',
 'MFI',
 'MINUS_DI',
 'MINUS_DM',
 'MOM',
 'PLUS_DI',
 'PLUS_DM',
 'PPO',
 'ROC',
 'ROCP',
 'ROCR',
 'ROCR100',
 'RSI',
 'STOCH',
 'STOCHF',
 'STOCHRSI',
 'TRIX',
 'ULTOSC',
 'WILLR']

In [ ]:
talib.get_functions()

['HT_DCPERIOD',
 'HT_DCPHASE',
 'HT_PHASOR',
 'HT_SINE',
 'HT_TRENDMODE',
 'ADD',
 'DIV',
 'MAX',
 'MAXINDEX',
 'MIN',
 'MININDEX',
 'MINMAX',
 'MINMAXINDEX',
 'MULT',
 'SUB',
 'SUM',
 'ACOS',
 'ASIN',
 'ATAN',
 'CEIL',
 'COS',
 'COSH',
 'EXP',
 'FLOOR',
 'LN',
 'LOG10',
 'SIN',
 'SINH',
 'SQRT',
 'TAN',
 'TANH',
 'ADX',
 'ADXR',
 'APO',
 'AROON',
 'AROONOSC',
 'BOP',
 'CCI',
 'CMO',
 'DX',
 'MACD',
 'MACDEXT',
 'MACDFIX',
 'MFI',
 'MINUS_DI',
 'MINUS_DM',
 'MOM',
 'PLUS_DI',
 'PLUS_DM',
 'PPO',
 'ROC',
 'ROCP',
 'ROCR',
 'ROCR100',
 'RSI',
 'STOCH',
 'STOCHF',
 'STOCHRSI',
 'TRIX',
 'ULTOSC',
 'WILLR',
 'BBANDS',
 'DEMA',
 'EMA',
 'HT_TRENDLINE',
 'KAMA',
 'MA',
 'MAMA',
 'MAVP',
 'MIDPOINT',
 'MIDPRICE',
 'SAR',
 'SAREXT',
 'SMA',
 'T3',
 'TEMA',
 'TRIMA',
 'WMA',
 'CDL2CROWS',
 'CDL3BLACKCROWS',
 'CDL3INSIDE',
 'CDL3LINESTRIKE',
 'CDL3OUTSIDE',
 'CDL3STARSINSOUTH',
 'CDL3WHITESOLDIERS',
 'CDLABANDONEDBABY',
 'CDLADVANCEBLOCK',
 'CDLBELTHOLD',
 'CDLBREAKAWAY',
 'CDLCLOSINGMARUBOZU',


### Backtesting Optimized

#### 1 (Ongoing)
This code has the necessary modifications and is used to further create functions

In [171]:
# Choosing currency pair
symbols = "EURUSDm"

# Retrieve currency data in 1 minute timeframe with a time range
ticks = mt5.copy_rates_range(symbols, mt5.TIMEFRAME_M5, datetime(2024,3,1), datetime(2024,3,30))
data = pd.DataFrame(ticks)
data['time'] = pd.to_datetime(data['time'], unit='s')  # convert time including seconds
data.rename(columns={"tick_volume": "volume"}, inplace=True)

# Get the "point" or tick of the pair for symbol presumably = 0.00001
sym_point = mt5.symbol_info(symbols).point

# Define pip values and spread adjustment, a point is 0.00001
take_profit_pips = 0.0005  # 5 pips for take profit
stop_loss_pips = 0.0001    # 1 pips for stop loss

# Sample DataFrame with close prices and moving averages (data)
# Calculate Moving Averages
data['MA_50'] = talib.SMA(data['close'], timeperiod=50)
data['MA_200'] = talib.SMA(data['close'], timeperiod=200)

# Initialize columns for trade entry prices, targets, and results
data['Entry_Price'] = None
data['Take_Profit'] = None
data['Stop_Loss'] = None
data['Trade'] = None  # 'Buy' or 'Sell'
data['Trade_Success'] = None
data['Pips'] = None

# Identify crossovers and calculate thresholds in one pass
buy_signals = (data['MA_50'] > data['MA_200']) & (data['MA_50'].shift(1) <= data['MA_200'].shift(1))
sell_signals = (data['MA_50'] < data['MA_200']) & (data['MA_50'].shift(1) >= data['MA_200'].shift(1))

# Buy signal: entry price + half spread, set TP and SL
data.loc[buy_signals, 'Entry_Price'] = data['close'] + data['Spread']
data.loc[buy_signals, 'Take_Profit'] = data['Entry_Price'] + take_profit_pips
data.loc[buy_signals, 'Stop_Loss'] = data['Entry_Price'] - stop_loss_pips
data.loc[buy_signals, 'Trade'] = 'Buy'

# Sell signal: entry price - half spread, set TP and SL
data.loc[sell_signals, 'Entry_Price'] = data['close'] - data['Spread']
data.loc[sell_signals, 'Take_Profit'] = data['Entry_Price'] - take_profit_pips
data.loc[sell_signals, 'Stop_Loss'] = data['Entry_Price'] + stop_loss_pips
data.loc[sell_signals, 'Trade'] = 'Sell'

# Check if TP or SL is hit in the next 1 to 200 periods for each trade row
for i, row in data.iterrows():
    if row['Trade'] in ['Buy', 'Sell']:
        trade_success = 'No Result'
        pips_result = 0  # Default pip result if no TP or SL is hit

        # Iterate through increasing periods (1 to 200) to check TP/SL
        for period in range(1, 1000):
            # Define the future high and low within the current period
            future_high = data['high'][i:i+period].max()
            future_low = data['low'][i:i+period].min()

            # Set conditions based on trade type
            if row['Trade'] == 'Buy':
                take_profit_hit = future_high >= row['Take_Profit']
                stop_loss_hit = future_low <= row['Stop_Loss']
            else:  # Sell
                take_profit_hit = future_low <= row['Take_Profit']
                stop_loss_hit = future_high >= row['Stop_Loss']

            # Determine if TP or SL is hit within this period
            if take_profit_hit:
                trade_success = 'Success'
                pips_result = take_profit_pips
                break  # Exit the loop once TP is hit
            elif stop_loss_hit:
                trade_success = 'Fail'
                pips_result = -1 * stop_loss_pips
                break  # Exit the loop once SL is hit

        # Update the DataFrame with the trade result
        data.at[i, 'Trade_Success'] = trade_success
        data.at[i, 'Pips'] = pips_result*10000

# Filter and view only the trades
trades_df = data.dropna(subset=['Trade'])

# Get the month name
month_name = datetime(2024,3,1).strftime("%b")

# Extract counts for each category
buy_sell = trades_df["Trade"].value_counts()
success_fail = trades_df["Trade_Success"].value_counts()
total_trades = trades_df.shape[0]   # For the total count of trades
sum_pips = trades_df.groupby("Trade_Success")["Pips"].sum().sum() # Sum of pips

# Combine values into a DataFrame with the desired structure
combined_df = pd.DataFrame({
    month_name: [
        total_trades,
        buy_sell.get('Buy', 0),
        buy_sell.get('Sell', 0),
        success_fail.get('Success', 0),
        success_fail.get('Fail', 0),
        sum_pips
        ]
}, index=["Trades","Buy", "Sell", "Success", "Fail", "Pips Earn/Lost"])

print(combined_df)



                 Mar
Trades          30.0
Buy             15.0
Sell            15.0
Success         12.0
Fail            18.0
Pips Earn/Lost  -6.0


In [60]:
trades_df[["time",'Trade','Trade_Success','Pips']].head()

,time,Trade,Trade_Success,Pips
593,2024-03-05 01:30:00,Sell,Fail,-1.0
762,2024-03-05 15:35:00,Buy,Success,1.0
879,2024-03-06 01:20:00,Sell,Fail,-1.0
955,2024-03-06 07:40:00,Buy,Success,1.0
1251,2024-03-07 08:20:00,Sell,Fail,-1.0


#### Functions


__NB__
- The "No Result" status indicates that neither the Take Profit (TP) nor the Stop Loss (SL) was hit within the available data range. This often occurs near the end of the dataset, for example, if the data covers one month, trades toward the end of that month are more likely to show "No Result." The same applies if the data range spans a year.

In [4]:
def get_year_dates(year):
    """
    Returns the start and end dates of a given year. If the given year is the current year,
    it returns the start of the year to the current date.

    Parameters:
        year (int): The year (e.g., 2024).

    Returns:
        tuple: A tuple containing the start and end dates as datetime objects.
    """
    from datetime import datetime

    # Get the current year and date
    current_date = datetime.now()
    current_year = current_date.year

    # Determine start and end dates
    start_date = datetime(year, 1, 1)
    if year == current_year:
        end_date = current_date
    else:
        end_date = datetime(year, 12, 31)

    return start_date, end_date


In [39]:
# Make the TP & SL a global variable
tp_amount = 4
sl_amount = 2

In [40]:
# Backtests for a whole year
def moving_average_backtest_year(symbol, timeframe, year, num1, num2):
    """
    Performs a moving average crossover backtest for a given symbol, timeframe, and date range.

    Parameters:
        symbol (str): The trading symbol (e.g., "EURUSDm").
        timeframe (int): The MetaTrader 5 timeframe constant (e.g., mt5.TIMEFRAME_M15).
        year (int): The year (e.g., 2024).
        num1 (int): The period for the first moving average.
        num2 (int): The period for the second moving average.

    Returns:
        dict: A dictionary summarizing trade statistics, including total trades, 
              the number of buy and sell trades, successful and failed trades, 
              and the total pips earned or lost.
    """
    # Get the dates
    start_date = get_year_dates(year)[0]
    end_date = get_year_dates(year)[1]

    # Retrieve currency data in 15 minute timeframe with a time range
    ticks = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    data = pd.DataFrame(ticks)
    data['time'] = pd.to_datetime(data['time'], unit='s')  # convert time including seconds
    data.rename(columns={"tick_volume": "volume"}, inplace=True)

    # Get the "point" or tick of the pair for symbol presumably = 0.00001
    sym_point = mt5.symbol_info(symbol).point
    
    # Define pip values and spread adjustment
    # At 0.01 pips 100 points = 10 pips = 1$
    take_profit_pips = tp_amount * (sym_point * 100)  # 10 pips, 5$ for take profit
    stop_loss_pips = sl_amount * (sym_point * 100)  # 10 pips, 1$ below entry

    
    # Create column name from num1 and num2
    name1 = "MA_" + str(num1)
    name2 = "MA_" + str(num2)

    # Sample DataFrame with close prices and moving averages (data)
    # Calculate Moving Averages
    data[name1] = talib.SMA(data['close'], timeperiod=num1)
    data[name2] = talib.SMA(data['close'], timeperiod=num2)

    # Initialize columns for trade entry prices, targets, and results
    data['Entry_Price'] = None
    data['Take_Profit'] = None
    data['Stop_Loss'] = None
    data['Trade'] = None  # 'Buy' or 'Sell'
    data['Trade_Success'] = None
    data['Pips'] = None

    # Identify crossovers and calculate thresholds in one pass
    buy_signals = (data[name1] > data[name2]) & (data[name1].shift(1) <= data[name2].shift(1))
    sell_signals = (data[name1] < data[name2]) & (data[name1].shift(1) >= data[name2].shift(1))

    # Buy signal: entry price + half spread, set TP and SL
    data.loc[buy_signals, 'Entry_Price'] = data['close'] + (data['spread']*sym_point)
    data.loc[buy_signals, 'Take_Profit'] = data['Entry_Price'] + take_profit_pips
    data.loc[buy_signals, 'Stop_Loss'] = data['Entry_Price'] - stop_loss_pips
    data.loc[buy_signals, 'Trade'] = 'Buy'

    # Sell signal: entry price - spread, set TP and SL
    data.loc[sell_signals, 'Entry_Price'] = data['close'] - (data['spread']*sym_point)
    data.loc[sell_signals, 'Take_Profit'] = data['Entry_Price'] - take_profit_pips
    data.loc[sell_signals, 'Stop_Loss'] = data['Entry_Price'] + stop_loss_pips
    data.loc[sell_signals, 'Trade'] = 'Sell'

    # Check if TP or SL is hit in the next 1 to 200 periods for each trade row
    for i, row in data.iterrows():
        if row['Trade'] in ['Buy', 'Sell']:
            trade_success = 'No Result'
            pips_result = 0  # Default pip result if no TP or SL is hit

            # Iterate through increasing periods (1 to 200) to check TP/SL
            for period in range(1, 1000):
                # Define the future high and low within the current period
                future_high = data['high'][i:i+period].max()
                future_low = data['low'][i:i+period].min()

                # Set conditions based on trade type
                if row['Trade'] == 'Buy':
                    take_profit_hit = future_high >= row['Take_Profit']
                    stop_loss_hit = future_low <= row['Stop_Loss']
                else:  # Sell
                    take_profit_hit = future_low <= row['Take_Profit']
                    stop_loss_hit = future_high >= row['Stop_Loss']
                
                # Determine if TP or SL is hit within this period
                if take_profit_hit:
                    trade_success = 'Success'
                    pips_result = tp_amount
                    break  # Exit the loop once TP is hit
                elif stop_loss_hit:
                    trade_success = 'Fail'
                    pips_result = -1 * sl_amount
                    break  # Exit the loop once SL is hit

            # Update the DataFrame with the trade result
            data.at[i, 'Trade_Success'] = trade_success
            data.at[i, 'Pips'] = pips_result

    # Filter and view only the trades
    trades_df = data.dropna(subset=['Trade'])


    # Extract counts for each category
    buy_sell = trades_df["Trade"].value_counts()
    success_fail = trades_df["Trade_Success"].value_counts()
    successful_trade = trades_df[trades_df["Trade_Success"]=="Success"].shape[0]
    total_trades = trades_df.shape[0]   # For the total count of trades
    sum_pips = trades_df.groupby("Trade_Success")["Pips"].sum().sum() # Sum of pips

    combined_dict = {
            "Trades": total_trades,
            "Buy": buy_sell.get('Buy', 0),
            "Sell": buy_sell.get('Sell', 0),
            "Success": success_fail.get('Success', 0),
            "Fail": success_fail.get('Fail', 0),
            "No Result": success_fail.get('No Result', 0),
            "Pips Earn/Lost": sum_pips,
            "Win Rate %": round((successful_trade/total_trades)*100, 0)
        }
    

    return combined_dict

In [41]:
# Backtests result shows for each month of the year
def moving_average_backtest_year_month(symbol, timeframe, year, num1, num2):
    """
    Performs a moving average crossover backtest for a given symbol, timeframe, and date range.

    Parameters:
        symbol (str): The trading symbol (e.g., "EURUSDm").
        timeframe (int): The MetaTrader 5 timeframe constant (e.g., mt5.TIMEFRAME_M15).
        year (int): The year (e.g., 2024).
        num1 (int): The period for the first moving average.
        num2 (int): The period for the second moving average.

    Returns:
        pd.DataFrame: A DataFrame summarizing monthly trade statistics.
    """
    # Get the dates
    start_date = get_year_dates(year)[0]
    end_date = get_year_dates(year)[1]

    # Retrieve currency data
    ticks = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    data = pd.DataFrame(ticks)
    data['time'] = pd.to_datetime(data['time'], unit='s')  # Convert time including seconds
    data.rename(columns={"tick_volume": "volume"}, inplace=True)

    # Get the "point" or tick of the pair for symbol presumably = 0.00001
    sym_point = mt5.symbol_info(symbol).point

    # Define pip values and spread adjustment
    take_profit_pips = tp_amount * (sym_point * 100)
    stop_loss_pips = sl_amount * (sym_point * 100)

    # Create column name from num1 and num2
    name1 = "MA_" + str(num1)
    name2 = "MA_" + str(num2)

    # Calculate Moving Averages
    data[name1] = talib.SMA(data['close'], timeperiod=num1)
    data[name2] = talib.SMA(data['close'], timeperiod=num2)

    # Initialize columns for trade entry prices, targets, and results
    data['Entry_Price'] = None
    data['Take_Profit'] = None
    data['Stop_Loss'] = None
    data['Trade'] = None  # 'Buy' or 'Sell'
    data['Trade_Success'] = None
    data['Pips'] = None

    # Identify crossovers
    buy_signals = (data[name1] > data[name2]) & (data[name1].shift(1) <= data[name2].shift(1))
    sell_signals = (data[name1] < data[name2]) & (data[name1].shift(1) >= data[name2].shift(1))

    # Assign trade details
    data.loc[buy_signals, 'Entry_Price'] = data['close'] + (data['spread'] * sym_point)
    data.loc[buy_signals, 'Take_Profit'] = data['Entry_Price'] + take_profit_pips
    data.loc[buy_signals, 'Stop_Loss'] = data['Entry_Price'] - stop_loss_pips
    data.loc[buy_signals, 'Trade'] = 'Buy'

    data.loc[sell_signals, 'Entry_Price'] = data['close'] - (data['spread'] * sym_point)
    data.loc[sell_signals, 'Take_Profit'] = data['Entry_Price'] - take_profit_pips
    data.loc[sell_signals, 'Stop_Loss'] = data['Entry_Price'] + stop_loss_pips
    data.loc[sell_signals, 'Trade'] = 'Sell'

    # Check trade outcomes
    for i, row in data.iterrows():
        if row['Trade'] in ['Buy', 'Sell']:
            trade_success = 'No Result'
            pips_result = 0

            for period in range(1, 1000):
                future_high = data['high'][i:i + period].max()
                future_low = data['low'][i:i + period].min()

                if row['Trade'] == 'Buy':
                    take_profit_hit = future_high >= row['Take_Profit']
                    stop_loss_hit = future_low <= row['Stop_Loss']
                else:
                    take_profit_hit = future_low <= row['Take_Profit']
                    stop_loss_hit = future_high >= row['Stop_Loss']

                if take_profit_hit:
                    trade_success = 'Success'
                    pips_result = tp_amount
                    break
                elif stop_loss_hit:
                    trade_success = 'Fail'
                    pips_result = -sl_amount
                    break

            data.at[i, 'Trade_Success'] = trade_success
            data.at[i, 'Pips'] = pips_result

    # Filter trades and add a 'Month' column
    trades_df = data.dropna(subset=['Trade']).copy()
    trades_df['Month'] = trades_df['time'].dt.month

    # Aggregate monthly statistics
    monthly_stats = trades_df.groupby('Month').agg(
        Trades=('Trade', 'count'),
        Buy=('Trade', lambda x: (x == 'Buy').sum()),
        Sell=('Trade', lambda x: (x == 'Sell').sum()),
        Success=('Trade_Success', lambda x: (x == 'Success').sum()),
        Fail=('Trade_Success', lambda x: (x == 'Fail').sum()),
        No_Result=('Trade_Success', lambda x: (x == 'No Result').sum()),
        Pips_Earned_Lost=('Pips', 'sum')
    ).reset_index()

    # Map month numbers to names
    monthly_stats['Month'] = monthly_stats['Month'].apply(lambda x: pd.Timestamp(f'2024-{x:02d}-01').strftime('%B'))

    return monthly_stats


In [42]:
def best_moving_averages(symbol, timeframe, year):
    """
    Backtest multiple moving average crossovers strategies for a given symbol, timeframe, and year.
    
    Args:
        symbol (str): The trading symbol (e.g., 'EURUSDm').
        timeframe: Timeframe for the backtest (e.g., mt5.TIMEFRAME_H2).
        year (int): Year for the backtest.
    
    FYI:
        This function will take some time and some memory power to work
    
    Returns:
        pd.DataFrame: A sorted DataFrame of the results.
    """
    # Define the ranges for short and long moving averages
    ma_short_range = range(2, 50)
    ma_long_range = range(2, 50)
    
    # Initialize an empty list to store results
    results = []

    for long in ma_long_range:
        for short in ma_short_range:
            if short >= long:  # Ensure short MA is less than long MA
                continue
            
            maby = moving_average_backtest_year(symbol, timeframe, year, short, long)
            successful_trades = maby['Success']
            total_trade = maby['Trades']
            failed_trades = maby['Fail']
            no_result_trades = maby['No Result']
            amount_sum = maby['Pips Earn/Lost']
            yr_win_rate = maby['Win Rate %']
            
            # Append the result as a dictionary
            results.append({
                "Symbol": symbol,
                "Year": year,
                "Short MA": short,
                "Long MA": long,
                "Total Trades": total_trade,
                "Successful Trades": successful_trades,
                "Failed Trades": failed_trades,
                "No Result Trades": no_result_trades,
                "Amount Earned/Lost": amount_sum,
                "Win Rate (%)": yr_win_rate
            })

    # Convert the list of results to a DataFrame
    results_df = pd.DataFrame(results)

    # Sort the DataFrame by 'Win Rate (%)' and 'Pips Earned/Lost' in descending order
    sorted_results_df = results_df.sort_values(
        by=["Win Rate (%)", "Amount Earned/Lost"],
        ascending=[False, False]
    )
    
    return sorted_results_df


In [44]:
symbol = "EURUSDm"
timeframe = mt5.TIMEFRAME_H4
bma_2024= best_moving_averages(symbol, timeframe, 2024)
bma_2024

,Symbol,Year,Short MA,Long MA,Total Trades,Successful Trades,Failed Trades,No Result Trades,Amount Earned/Lost,Win Rate (%)
1052,EURUSDm,2024,19,48,29,15,14,0,32,52.0
1008,EURUSDm,2024,20,47,31,15,16,0,28,48.0
1097,EURUSDm,2024,18,49,29,14,15,0,26,48.0
921,EURUSDm,2024,20,45,34,15,19,0,22,44.0
1107,EURUSDm,2024,28,49,34,15,19,0,22,44.0
...,...,...,...,...,...,...,...,...,...,...
1121,EURUSDm,2024,42,49,46,7,39,0,-50,15.0
1076,EURUSDm,2024,43,48,54,8,46,0,-60,15.0
459,EURUSDm,2024,26,32,62,9,53,0,-70,15.0
496,EURUSDm,2024,2,34,93,14,79,0,-102,15.0


In [29]:
symbol = "EURUSDm"
timeframe = mt5.TIMEFRAME_M10
year = 2024
mab= moving_average_backtest_year_month(symbol, timeframe, year, 49, 71)
mab

,Month,Trades,Buy,Sell,Success,Fail,No_Result,Pips_Earned_Lost
0,January,54,27,27,15,39,0,-18
1,February,52,26,26,14,38,0,-20
2,March,44,22,22,12,32,0,-16
3,April,54,27,27,16,38,0,-12
4,May,57,29,28,21,36,0,12
5,June,50,25,25,20,30,0,20
6,July,51,25,26,14,37,0,-18
7,August,54,27,27,19,35,0,6
8,September,64,32,32,20,44,0,-8
9,October,59,30,29,19,40,0,-4


__Info__
- Now we need to find the moving average pairs with the best average win rate (%) across the years

##### Testing different timeframes

H2

In [ ]:
symbol = "EURUSDm"
timeframe = mt5.TIMEFRAME_H2

# List of years to process
years = range(2019, 2025)

# Initialize an empty list to store results for each year
results = []

# Track progress with tqdm
for year in tqdm(years, desc="Processing Years", unit="year"):
    result = best_moving_averages(symbol, timeframe, year)
    results.append(result)

# Combine all dataframes into one
combined_results = pd.concat(results)

# Group by 'Short MA' and 'Long MA' and calculate the average win rate and total trades
average_win_rates = combined_results.groupby(['Short MA', 'Long MA']).agg(
    {'Win Rate (%)': 'mean', 'Total Trades': 'mean', "Failed Trades": 'mean', "Pips Earned/Lost": 'mean'}
).reset_index()

# Rename columns for clarity if needed
average_win_rates.rename(columns={
    'Win Rate (%)': 'Avg Yearly Win Rate (%)',
    'Total Trades': 'Avg Yearly Total Trades',
    "Failed Trades": 'Avg Yearly Failed Trades',
    "Pips Earned/Lost": "Average Pips Earned/Lost"
}, inplace=True)

# Sort by 'Average Win Rate (%)' in descending order
best_pairs_h2 = average_win_rates.sort_values(by='Avg Yearly Win Rate (%)', ascending=False)

# Convert all columns in the dataframe to integers
best_pairs_h2 = best_pairs_h2.astype(int)

best_pairs_h2.head()


For H1

In [11]:
symbol = "EURUSDm"
timeframe = mt5.TIMEFRAME_H1

# List of years to process
years = range(2014, 2025)

# Initialize an empty list to store results for each year
results = []

# Track progress with tqdm
for year in tqdm(years, desc="Processing Years", unit="year"):
    result = best_moving_averages(symbol, timeframe, year)
    results.append(result)

# Combine all dataframes into one
combined_results = pd.concat(results)

# Group by 'Short MA' and 'Long MA' and calculate the average win rate and total trades
average_win_rates = combined_results.groupby(['Short MA', 'Long MA']).agg(
    {'Win Rate (%)': 'mean', 'Total Trades': 'mean', "Failed Trades": 'mean', "Pips Earned/Lost": 'mean'}
).reset_index()

# Rename columns for clarity if needed
average_win_rates.rename(columns={
    'Win Rate (%)': 'Avg Yearly Win Rate (%)',
    'Total Trades': 'Avg Yearly Total Trades',
    "Failed Trades": 'Avg Yearly Failed Trades',
    "Pips Earned/Lost": "Average Pips Earned/Lost"
}, inplace=True)

# Sort by 'Average Win Rate (%)' in descending order
best_pairs_h1 = average_win_rates.sort_values(by='Avg Yearly Win Rate (%)', ascending=False)

# Convert all columns in the dataframe to integers
best_pairs_h1 = best_pairs_h1.astype(int)

best_pairs_h1.head()


Processing Years: 100%|██████████| 11/11 [1:20:39<00:00, 439.94s/year]


,Short MA,Long MA,Avg Yearly Win Rate (%),Avg Yearly Total Trades,Avg Yearly Failed Trades,Average Pips Earned/Lost
803,24,25,62,425,218,819
723,21,23,60,331,177,593
663,19,20,59,499,270,873
664,19,21,59,350,191,604
750,22,23,59,448,236,824


__Info__
- We have 252 trading days in a year (This basically removes weekends and certaion holidays).

_Remember_
- MT5 has a data retrieval limit of 50,000 records, starting from the current date and time. The amount of historical data you can access in terms of years depends on the timeframe you select. Shorter timeframes (e.g., 1-minute) will cover fewer years, while longer timeframes (e.g., 1-day) will cover more years within the 50,000-record limit.
- I'm yet to find a way to bypass this restriction but for now here is the timeframes and data limits
- _Also General data starts from_ ___Jan 2014___
    - 1-minute (M1) ≈ 1.5 months
    - 5-minute (M5) ≈ 8.7 months
    - 15-minute (M15) ≈ 2.2 years
    - 30-minute (M30) ≈ 4.4 years
    - 1-hour (H1) ≈ 8.7 years
    - 2-hour (H2) ≈ 17.4 years
    - 4-hour (H4) ≈ 34.8 years
    - 6-hour (H6) ≈ 52.2 years
    - 8-hour (H8) ≈ 69.6 years
    - 12-hour (H12) ≈ 103.7 years
    - 1-day (D1) ≈ 136.9 years
    - 1-week (W1) ≈ 961.5 years
    - 1-month (MN) ≈ 4166.7 years


##### Function for more data (optional)

In [42]:
from datetime import datetime, timedelta

def extract_data_by_day(symbol, timeframe, start_date, end_date):
    """
    Extract historical data from MetaTrader 5 by iterating through each day
    between start_date and end_date, appending data for each day.

    Parameters:
        symbol (str): The trading symbol (e.g., "EURUSD").
        timeframe (int): The MetaTrader 5 timeframe constant (e.g., mt5.TIMEFRAME_M1).
        start_date (datetime): The start date for data extraction.
        end_date (datetime): The end date for data extraction.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted data.
    """
    all_data = []  # List to store data for each day

    # Start the loop from start_date to end_date
    current_date = start_date
    while current_date <= end_date:
        next_day = current_date + timedelta(days=1)  # Move to the next day
        
        # Fetch data for the current day
        day_data = mt5.copy_rates_range(symbol, timeframe, current_date, next_day)
        
        if day_data is None or len(day_data) == 0:
            pass
        else:
            # Append the data for the current day to the list
            all_data.append(pd.DataFrame(day_data))
        
        # Move to the next day
        current_date = next_day

    # Combine all the data into a single DataFrame
    if all_data:
        final_data = pd.concat(all_data, ignore_index=True)
        final_data['time'] = pd.to_datetime(final_data['time'], unit='s')
        return final_data
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no data was found



yeah no data was found at certain timeframes at certain years

In [36]:
# Define the timeframe and date range
timeframe = mt5.TIMEFRAME_M10
start_date = datetime(2021, 1, 1)
end_date = datetime(2021, 2, 20)

data = pd.DataFrame(mt5.copy_rates_range("EURUSDm", timeframe, start_date, end_date))

# data = extract_data_by_day("EURUSDm", timeframe, start_date, end_date)
data

,time,open,high,low,close,tick_volume,spread,real_volume
0,1647502800,1.1032,1.10395,1.10297,1.10386,265,8,0


### Charts

In [175]:
data = mab[1]
data.head(1)

,time,open,high,low,close,volume,spread,real_volume,MA_24,MA_25,Entry_Price,Take_Profit,Stop_Loss,Trade,Trade_Success,Pips
0,2024-01-01 22:00:00,1.10268,1.10369,1.10268,1.10306,1195,127,0,NaN,NaN,None,None,None,None,None,None


In [176]:
view_trades= mab[1][['time','Trade','Trade_Success','Entry_Price','Take_Profit','Stop_Loss']]
view_trades.groupby(by=['Trade','Trade_Success']).count()

time  Entry_Price  Take_Profit  Stop_Loss
Trade Trade_Success                                           
Buy   Fail             88           88           88         88
      No Result         1            1            1          1
      Success          67           67           67         67
Sell  Fail             81           81           81         81
      Success          74           74           74         74

In [177]:
view_trades[view_trades['Trade_Success']=='No Result']

,time,Trade,Trade_Success,Entry_Price,Take_Profit,Stop_Loss
3079,2024-12-20 18:00:00,Buy,No Result,1.04452,1.04952,1.03952


In [160]:
# Specifying the area
data_plot = data.loc[220:].copy()

# Create the figure
fig = go.Figure()

# Add the candlestick trace
fig.add_trace(go.Candlestick(
    x=data_plot['time'],
    open=data_plot['open'],
    high=data_plot['high'],
    low=data_plot['low'],
    close=data_plot['close'],
    name="Candlestick"
))

# Add moving average for MA_16
fig.add_trace(go.Scatter(
    x=data_plot['time'],
    y=data_plot['MA_24'],
    mode='lines',
    line=dict(color='blue', width=1),
    name="MA_16"
))

# Add moving average for MA_50
fig.add_trace(go.Scatter(
    x=data_plot['time'],
    y=data_plot['MA_25'],
    mode='lines',
    line=dict(color='red', width=1),
    name="MA_50"
))

# Update layout for better readability
fig.update_layout(
    title="Candlestick Chart with Moving Averages",
    xaxis_title="Time",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

# Show the figure
fig.show()

# view trades
# print(view_trades)

In [159]:
view_trades[view_trades['Trade_Success']=='No Result']

,time,Trade,Trade_Success,Entry_Price,Take_Profit,Stop_Loss
239,2024-01-29 20:00:00,Sell,No Result,1.08312,1.07812,1.08812
244,2024-01-30 06:00:00,Buy,No Result,1.08232,1.08732,1.07732
245,2024-01-30 08:00:00,Sell,No Result,1.08253,1.07753,1.08753


In [155]:
view_trades.loc[220:250][['time','Trade','Trade_Success','Entry_Price','Take_Profit','Stop_Loss']]

,time,Trade,Trade_Success,Entry_Price,Take_Profit,Stop_Loss
220,2024-01-26 06:00:00,Sell,Fail,1.08181,1.07681,1.08681
221,2024-01-26 08:00:00,None,None,None,None,None
222,2024-01-26 10:00:00,None,None,None,None,None
223,2024-01-26 12:00:00,None,None,None,None,None
224,2024-01-26 14:00:00,None,None,None,None,None
225,2024-01-26 16:00:00,None,None,None,None,None
226,2024-01-26 18:00:00,None,None,None,None,None
227,2024-01-26 20:00:00,None,None,None,None,None
228,2024-01-28 22:00:00,None,None,None,None,None
229,2024-01-29 00:00:00,None,None,None,None,None


In [63]:
symbol_info= mt5.symbol_info("BTCUSDm")
point = symbol_info.point
point

0.01

In [59]:
mt5.symbol_info(symbol).spread * point

28.8

In [65]:
1 * symbol_info.point * 10

0.1

In [71]:
symbol_info._asdict()

{'custom': False,
 'chart_mode': 0,
 'select': True,
 'visible': True,
 'session_deals': 0,
 'session_buy_orders': 0,
 'session_sell_orders': 0,
 'volume': 0,
 'volumehigh': 0,
 'volumelow': 0,
 'time': 1734250622,
 'digits': 2,
 'spread': 2880,
 'spread_float': True,
 'ticks_bookdepth': 0,
 'trade_calc_mode': 5,
 'trade_mode': 4,
 'start_time': 0,
 'expiration_time': 0,
 'trade_stops_level': 0,
 'trade_freeze_level': 0,
 'trade_exemode': 2,
 'swap_mode': 1,
 'swap_rollover3days': 5,
 'margin_hedged_use_leg': False,
 'expiration_mode': 15,
 'filling_mode': 3,
 'order_mode': 127,
 'order_gtc_mode': 0,
 'option_mode': 0,
 'option_right': 0,
 'bid': 101740.99,
 'bidhigh': 102806.43,
 'bidlow': 101213.18,
 'ask': 101769.79,
 'askhigh': 102839.26,
 'asklow': 101241.98,
 'last': 0.0,
 'lasthigh': 0.0,
 'lastlow': 0.0,
 'volume_real': 0.0,
 'volumehigh_real': 0.0,
 'volumelow_real': 0.0,
 'option_strike': 0.0,
 'point': 0.01,
 'trade_tick_value': 0.01,
 'trade_tick_value_profit': 0.01,
 'trad

### Conclusion

- There were no substantial results within the 2 to 50 range with a 1:1 risk to reward ratio from 300 to 1000 pips